# Personalize Workshop Cleanup

This notebook will walk through deleting all of the resources created by the [Personalize workshop](./1.1-Personalize.ipynb). You should only need to perform these steps if you have deployed the Retail Demo Store in your own AWS account and want to deprovision the Personalize resources. If you are participating in an AWS-led workshop, this process is likely not necessary.

Resources have to deleted in a specific sequence to avoid dependency errors. In order, we will delete campaigns, solutions, event trackers, datasets, and the dataset group. In addition, we need to make sure that each resource type is fully deleted before moving on to the next resource type. We'll also delete the schemas for our datasets and the IAM roles we created for Personalize.

## Import Dependencies

To get started, let's import the dependencies we'll need for this notebook.

In [1]:
# Import Dependencies
import boto3
import time

from botocore.exceptions import ClientError

# Setup Clients
personalize = boto3.client('personalize')
ssm = boto3.client('ssm')

## Lookup Dataset Group ARN

Since we only want to delete resources created within the dataset group, let's lookup the dataset group ARN for our dataset group.

In [2]:
# Name of dataset group that was created in the Personalize workshop notebook.
dataset_group_name = 'retaildemostore'
dataset_group_arn = None

dataset_groups_paginator = personalize.get_paginator('list_dataset_groups')
for dataset_groups_page in dataset_groups_paginator.paginate():
    for dataset_group in dataset_groups_page['datasetGroups']:
        if dataset_group['name'] == dataset_group_name:
            dataset_group_arn = dataset_group['datasetGroupArn']
            break
            
assert dataset_group_arn is not None, 'Could not find dataset group; does it exist?'

print('Dataset Group Arn: ' + dataset_group_arn)

Dataset Group Arn: arn:aws:personalize:us-east-1:029498593638:dataset-group/retaildemostore


## Determine Solutions

Next let's gather the solution ARNs for all solutions in our dataset group.

In [3]:
solution_arns = []

solutions_response = personalize.list_solutions(datasetGroupArn = dataset_group_arn, maxResults = 100)
if 'solutions' in solutions_response:
    for solution in solutions_response['solutions']:
        solution_arns.append(solution['solutionArn'])
        
print('Solutions found: ' + str(solution_arns))

Solutions found: ['arn:aws:personalize:us-east-1:029498593638:solution/retaildemostore-product-personalization', 'arn:aws:personalize:us-east-1:029498593638:solution/retaildemostore-personalized-ranking', 'arn:aws:personalize:us-east-1:029498593638:solution/retaildemostore-related-products']


## Delete Campaigns for Solutions

For each solution, let's delete all associated campaigns.

In [4]:
campaign_arns_deleted = []

for solution_arn in solution_arns:
    campaigns_response = personalize.list_campaigns(solutionArn = solution_arn, maxResults = 100)
    
    if 'campaigns' in campaigns_response:
        for campaign in campaigns_response['campaigns']:
            print('Deleting campaign: ' + campaign['campaignArn'])
            
            personalize.delete_campaign(campaignArn = campaign['campaignArn'])
            campaign_arns_deleted.append(campaign['campaignArn'])

if len(campaign_arns_deleted) > 0:
    print('Done initiating deletes for {} campaign(s)'.format(len(campaign_arns_deleted)))
else:
    print('No campaigns to delete')

Deleting campaign: arn:aws:personalize:us-east-1:029498593638:campaign/retaildemostore-product-personalization
Deleting campaign: arn:aws:personalize:us-east-1:029498593638:campaign/retaildemostore-personalized-ranking
Deleting campaign: arn:aws:personalize:us-east-1:029498593638:campaign/retaildemostore-related-products
Done initiating deletes for 3 campaign(s)


### Clear SSM Parameters

Since the Retail Demo Store Recommendations service uses SSM parameters to determine the campaigns to use for product recommendations and search personalization, let's clear those parameters so we don't break the UI.

In [5]:
print('Clearing related products campaign arn SSM parameter')
ssm.put_parameter(
    Name='retaildemostore-related-products-campaign-arn',
    Description='Retail Demo Store Related Products Campaign Arn Parameter',
    Value='NONE',
    Type='String',
    Overwrite=True
)
print('Clearing product recommendation campaign arn SSM parameter')
ssm.put_parameter(
    Name='retaildemostore-product-recommendation-campaign-arn',
    Description='Retail Demo Store Product Recommendation Campaign Arn Parameter',
    Value='NONE',
    Type='String',
    Overwrite=True
)
print('Clearing personalized ranking campaign arn SSM parameter')
response = ssm.put_parameter(
    Name='retaildemostore-personalized-ranking-campaign-arn',
    Description='Retail Demo Store Personalized Ranking Campaign Arn Parameter',
    Value='NONE',
    Type='String',
    Overwrite=True
)

Clearing related products campaign arn SSM parameter
Clearing product recommendation campaign arn SSM parameter
Clearing personalized ranking campaign arn SSM parameter


### Wait for Campaigns to be Deleted

In [6]:
print('Waiting for {} campaigns to be fully deleted'.format(len(campaign_arns_deleted)))
while len(campaign_arns_deleted) > 0:
    try:
        status = None
        max_time = time.time() + 3*60*60 # 3 hours
        while time.time() < max_time:
            describe_campaign_response = personalize.describe_campaign(
                campaignArn = campaign_arns_deleted[0]
            )
            status = describe_campaign_response["campaign"]["status"]
            print('Campaign {}: {}'.format(campaign_arns_deleted[0], status))
            time.sleep(10)
            
        print('Exceeded wait time for campaign {} to delete; skipping'.format(campaign_arns_deleted[0]))
        campaign_arns_deleted.pop(0)

    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print('Campaign {} no longer exists'.format(campaign_arns_deleted[0]))
            campaign_arns_deleted.pop(0)
            
print('Done deleting campaigns')

Waiting for 3 campaigns to be fully deleted
Campaign arn:aws:personalize:us-east-1:029498593638:campaign/retaildemostore-product-personalization: DELETE PENDING
Campaign arn:aws:personalize:us-east-1:029498593638:campaign/retaildemostore-product-personalization: DELETE IN_PROGRESS
Campaign arn:aws:personalize:us-east-1:029498593638:campaign/retaildemostore-product-personalization: DELETE IN_PROGRESS
Campaign arn:aws:personalize:us-east-1:029498593638:campaign/retaildemostore-product-personalization: DELETE IN_PROGRESS
Campaign arn:aws:personalize:us-east-1:029498593638:campaign/retaildemostore-product-personalization: DELETE IN_PROGRESS
Campaign arn:aws:personalize:us-east-1:029498593638:campaign/retaildemostore-product-personalization: DELETE IN_PROGRESS
Campaign arn:aws:personalize:us-east-1:029498593638:campaign/retaildemostore-product-personalization: DELETE IN_PROGRESS
Campaign arn:aws:personalize:us-east-1:029498593638:campaign/retaildemostore-product-personalization: DELETE IN_P

## Delete Solutions

With the campaigns fully deleted, we can now delete the solutions for our dataset group.

In [7]:
solution_arns_deleted = []

for solution_arn in solution_arns:
    print('Deleting solution: ' + solution_arn)

    personalize.delete_solution(solutionArn = solution_arn)
    solution_arns_deleted.append(solution_arn)

if len(solution_arns_deleted) > 0:
    print('Done initiating deletes for {} solution(s)'.format(len(solution_arns_deleted)))
else:
    print('No solutions to delete')

Deleting solution: arn:aws:personalize:us-east-1:029498593638:solution/retaildemostore-product-personalization
Deleting solution: arn:aws:personalize:us-east-1:029498593638:solution/retaildemostore-personalized-ranking
Deleting solution: arn:aws:personalize:us-east-1:029498593638:solution/retaildemostore-related-products
Done initiating deletes for 3 solution(s)


### Wait for Solutions to be Deleted

Before we can move on, ensure the solutions have been fully deleted.

In [8]:
print('Waiting for {} solutions to be fully deleted'.format(len(solution_arns_deleted)))
while len(solution_arns_deleted) > 0:
    try:
        status = None
        max_time = time.time() + 3*60*60 # 3 hours
        while time.time() < max_time:
            describe_solution_response = personalize.describe_solution(
                solutionArn = solution_arns_deleted[0]
            )
            status = describe_solution_response["solution"]["status"]
            print('Solution {}: {}'.format(solution_arns_deleted[0], status))
            time.sleep(10)
            
        print('Exceeded wait time for solution {} to delete; skipping'.format(solution_arns_deleted[0]))
        solution_arns_deleted.pop(0)

    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print('Solution {} no longer exists'.format(solution_arns_deleted[0]))
            solution_arns_deleted.pop(0)
            
print('Done deleting solutions')

Waiting for 3 solutions to be fully deleted
Solution arn:aws:personalize:us-east-1:029498593638:solution/retaildemostore-product-personalization: DELETE PENDING
Solution arn:aws:personalize:us-east-1:029498593638:solution/retaildemostore-product-personalization: DELETE PENDING
Solution arn:aws:personalize:us-east-1:029498593638:solution/retaildemostore-product-personalization: DELETE IN_PROGRESS
Solution arn:aws:personalize:us-east-1:029498593638:solution/retaildemostore-product-personalization no longer exists
Solution arn:aws:personalize:us-east-1:029498593638:solution/retaildemostore-personalized-ranking no longer exists
Solution arn:aws:personalize:us-east-1:029498593638:solution/retaildemostore-related-products no longer exists
Done deleting solutions


## Delete Event Trackers

Next we'll delete any event trackers.

In [9]:
print('Deleting event trackers for dataset group')
event_trackers = 0
event_trackers_paginator = personalize.get_paginator('list_event_trackers')
for event_tracker_page in event_trackers_paginator.paginate(datasetGroupArn = dataset_group_arn):
    for event_tracker in event_tracker_page['eventTrackers']:
        print('Deleting event tracker {}'.format(event_tracker['eventTrackerArn']))
        personalize.delete_event_tracker(eventTrackerArn = event_tracker['eventTrackerArn'])
        event_trackers += 1
        
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time and event_trackers > 0:
    print('Waiting for event trackers to be deleted...')
    time.sleep(10)
    event_trackers = 0
    event_trackers_paginator = personalize.get_paginator('list_event_trackers')
    for event_tracker_page in event_trackers_paginator.paginate(datasetGroupArn = dataset_group_arn):
        event_trackers += len(event_tracker_page['eventTrackers'])
        
print('Done deleting event trackers')

Deleting event trackers for dataset group
Deleting event tracker arn:aws:personalize:us-east-1:029498593638:event-tracker/700b2f9d
Waiting for event trackers to be deleted...
Done deleting event trackers


### Clear Event Tracker SSM Parameter

Just as with the campaign ARNs, we need to clear the SSM parameter for our event tracker ID. **However, in order for the Retail Demo Store Web UI to stop sending events to this event tracker, a rebuild of the Web UI service must be triggered in CodePipeline.**

In [10]:
ssm.put_parameter(
    Name='retaildemostore-personalize-event-tracker-id',
    Description='Retail Demo Store Personalize Event Tracker ID Parameter',
    Value='NONE',
    Type='String',
    Overwrite=True
)

{'Version': 3,
 'Tier': 'Standard',
 'ResponseMetadata': {'RequestId': '6fef2a55-0af1-49b1-80a9-c093d2658ed7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 16 Sep 2020 02:48:07 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '31',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6fef2a55-0af1-49b1-80a9-c093d2658ed7'},
  'RetryAttempts': 0}}

## Delete Datasets

Next, we can delete the datasets for our dataset group.

In [11]:
print('Deleting datasets for dataset group')
dataset_arns_deleted = []
dataset_paginator = personalize.get_paginator('list_datasets')

for dataset_page in dataset_paginator.paginate(datasetGroupArn = dataset_group_arn):
    for dataset in dataset_page['datasets']:
        print('Deleting dataset {}'.format(dataset['datasetArn']))
        personalize.delete_dataset(datasetArn = dataset['datasetArn'])
        dataset_arns_deleted.append(dataset['datasetArn'])
        
if len(dataset_arns_deleted) > 0:
    print('Done initiating deletes for {} datasets(s)'.format(len(dataset_arns_deleted)))
else:
    print('No datasets to delete')

Deleting datasets for dataset group
Deleting dataset arn:aws:personalize:us-east-1:029498593638:dataset/retaildemostore/INTERACTIONS
Deleting dataset arn:aws:personalize:us-east-1:029498593638:dataset/retaildemostore/ITEMS
Deleting dataset arn:aws:personalize:us-east-1:029498593638:dataset/retaildemostore/USERS
Done initiating deletes for 3 datasets(s)


### Wait for Datasets to be Deleted

In [12]:
print('Waiting for {} datasets to be fully deleted'.format(len(dataset_arns_deleted)))
while len(dataset_arns_deleted) > 0:
    try:
        status = None
        max_time = time.time() + 3*60*60 # 3 hours
        while time.time() < max_time:
            describe_dataset_response = personalize.describe_dataset(
                datasetArn = dataset_arns_deleted[0]
            )
            status = describe_dataset_response["dataset"]["status"]
            print('Dataset {}: {}'.format(dataset_arns_deleted[0], status))
            time.sleep(10)
            
        print('Exceeded wait time for dataset {} to delete; skipping'.format(dataset_arns_deleted[0]))
        dataset_arns_deleted.pop(0)

    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print('Dataset {} no longer exists'.format(dataset_arns_deleted[0]))
            dataset_arns_deleted.pop(0)
            
print('Done deleting datasets')

Waiting for 3 datasets to be fully deleted
Dataset arn:aws:personalize:us-east-1:029498593638:dataset/retaildemostore/INTERACTIONS: DELETE PENDING
Dataset arn:aws:personalize:us-east-1:029498593638:dataset/retaildemostore/INTERACTIONS no longer exists
Dataset arn:aws:personalize:us-east-1:029498593638:dataset/retaildemostore/ITEMS no longer exists
Dataset arn:aws:personalize:us-east-1:029498593638:dataset/retaildemostore/USERS no longer exists
Done deleting datasets


## Delete Dataset Group

Finally, we can delete our dataset group.

In [13]:
print('Deleting dataset group')
personalize.delete_dataset_group(datasetGroupArn = dataset_group_arn)

Deleting dataset group


{'ResponseMetadata': {'RequestId': 'ba20361e-628b-4753-8fa6-fb73c2e93689',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Wed, 16 Sep 2020 02:48:17 GMT',
   'x-amzn-requestid': 'ba20361e-628b-4753-8fa6-fb73c2e93689',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

### Wait for Dataset Group to be Deleted

In [14]:
print('Waiting for dataset group to be fully deleted')
try: 
    status = None
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        describe_dataset_group_response = personalize.describe_dataset_group(
            datasetGroupArn = dataset_group_arn
        )
        status = describe_dataset_group_response["datasetGroup"]["status"]
        print("Dataset group: {}".format(status))
        time.sleep(10)
        
except ClientError as e:
    error_code = e.response['Error']['Code']
    if error_code == 'ResourceNotFoundException':
        print("Dataset group fully deleted")

Waiting for dataset group to be fully deleted
Dataset group: DELETE PENDING
Dataset group: DELETE IN_PROGRESS
Dataset group: DELETE IN_PROGRESS
Dataset group: DELETE IN_PROGRESS
Dataset group fully deleted


## Delete Schemas

We're almost done. The last step to cleaning up Personalize resources is to delete the schemas for our datasets.

In [15]:
schemas_to_delete = [ 'retaildemostore-schema-users', 'retaildemostore-schema-items', 'retaildemostore-schema-interactions' ]
schema_paginator = personalize.get_paginator('list_schemas')
for schema_page in schema_paginator.paginate():
    for schema in schema_page['schemas']:
        if schema['name'] in schemas_to_delete:
            print('Deleting schema {}'.format(schema['schemaArn']))
            personalize.delete_schema(schemaArn = schema['schemaArn'])
            
print('Done deleting schemas')

Deleting schema arn:aws:personalize:us-east-1:029498593638:schema/retaildemostore-schema-interactions
Deleting schema arn:aws:personalize:us-east-1:029498593638:schema/retaildemostore-schema-items
Deleting schema arn:aws:personalize:us-east-1:029498593638:schema/retaildemostore-schema-users
Done deleting schemas


## Delete IAM Role

Finally, we'll cleanup the IAM role we created to allow Personalize to access the S3 bucket where we placed our CSVs.

In [16]:
!aws iam detach-role-policy --role-name RetailDemoStorePersonalizeS3Role --policy-arn arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess
!aws iam delete-role --role-name RetailDemoStorePersonalizeS3Role

## Cleanup Complete

All resources created by the Personalize workshop have been deleted.